<a href="https://colab.research.google.com/github/FaizalSriJamal/python/blob/main/Linear%20Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Files Needed

Run the following 2 code cells to download and unzip the required data. \
You do not have to change anything in these 2 cells.

In [ ]:
import requests

filename = 'data-week-12.zip'
url = f'https://d17lzt44idt8rf.cloudfront.net/{filename}'
response = requests.get(url)

# Make sure the request was successful
if response.status_code == 200:

  # Write the content to a file
  with open(filename, 'wb') as f:
      f.write(response.content)

In [ ]:
!unzip $filename

Archive:  data-week-12.zip
replace 00_raw_data/hdb_block_info.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Table of Contents


- [Import Packages & Notebook Configurations](#Import-Packages-&-Notebook-Configurations)
- [Loading Data and Quick Checking](#Loading-Data-and-Quick-Checking)
  - [Loading Data](#Loading-Data)
  - [Quick Checking](#Quick-Checking)
- [Features Engineering](#Features-Engineering)
    - [Using apply() function](#Using-apply()-function)
    - [Using pandas vectorized functions](#Using-pandas-vectorized-functions)
- [Data Wrangling](#Data-Wrangling)
- [Exploratory Data Analysis](#Exploratory-Data-Analysis)
  - [Correlation](#Correlation)
    - [Breakdown and Understand the Pairplot](#Breakdown-and-Understand-the-Pairplot)
    - [Scatterplot for Visualizing Relationship between 2 Variables](#Scatterplot-for-Visualizing-Relationship-between-2-Variables)
  - [Descriptive Statistics](#Descriptive-Statistics)
- [Modeling with Linear Regression](#Modeling-with-Linear-Regression)
  - [Model Selection](#Model-Selection)
  - [Separating Train and Test Data sets](#Separating-Train-and-Test-Data-sets)
    - [Key Step 1: Instantiate the Model](#Key-Step-1:-Instantiate-the-Model)
    - [Key Step 2: Train the Model](#Key-Step-2:-Train-the-Model)
    - [Key Step 3: Predict based on new inputs](#Key-Step-3:-Predict-based-on-new-inputs)
    - [Key Step 4 (New): Validate the the Predicted Values](#Key-Step-4-(New):-Validate-the-the-Predicted-Values)
    - [Key 5 (New): Understand the Predictors](#Key-5-(New):-Understand-the-Predictors)
- [More Hands-on Practices: Build More Models](#More-Hands-on-Practices:-Build-More-Models)
  - [Model B](#Model-B)
  - [Model C](#Model-C)

# Import Packages & Notebook Configurations

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Settings for Matplotlib (& Seaborn)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Import libraries for charting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Set the size of charts
plt.rc('figure', figsize=(16,9))
sns.set_context(context={'figure.figsize': (16,9)})

![ml_workflow](https://i.imgur.com/xf43rGO.png)

---

# Loading Data and Quick Checking

## Loading Data

In [ ]:
df_hdb = pd.read_excel('00_raw_data/hdb_resale_basic.xlsx')

In [ ]:
df_fac = pd.read_excel('00_raw_data/hdb_facilities_distance.xlsx')

## Quick Checking

In [ ]:
df_hdb.info()

In [ ]:
df_fac.info()

In [ ]:
# Quick Stats for Numerical Columns
df_fac.describe()

In [ ]:
# Check Categorical Columns
df_hdb['town'].value_counts(normalize=True) #in percentage

In [ ]:
# Check Categorical Columns
df_hdb['Tranc_Year'].value_counts(normalize=True)

# Features Engineering

### Using apply() function

In [ ]:
def convert_squared_meters_to_feet(row):
    m2_to_feet2 = 10.7639
    unit_floor_area_sqft = row['floor_area_sqm'] * m2_to_feet2
    return round(unit_floor_area_sqft, 2)

In [ ]:
df_hdb['floor_area_sqft'] = df_hdb.apply(convert_squared_meters_to_feet, axis=1)

In [ ]:
df_hdb = df_hdb.drop('floor_area_sqft', axis=1)

### Using pandas vectorized functions

In [ ]:
df_hdb['floor_area_sqft'] = df_hdb['floor_area_sqm'] * 10.7639

# Data Wrangling

🔷**TASK**

Create a new column **price_per_sqft**

In [ ]:
df_hdb['price_per_sqft'] = df_hdb['resale_price'] / df_hdb['floor_area_sqft']
df_hdb

🔷**TASK**

Create a new column **flat_type_numerised** in **hdbdf** to store the values based on the mapping given below:

![](https://i.imgur.com/PL4d2tT.png)


In [ ]:
def flat_type_to_num(row):
  room = row['flat_type']
  if room == '1 ROOM':
    return 1
  elif room == '2 ROOM':
    return 2
  elif room == '3 ROOM':
    return 3
  elif room == '4 ROOM':
    return 4
  elif room == '5 ROOM':
    return 5
  elif room == 'EXECUTIVE':
    return 6
  else :
    return 7



df_hdb['flat_type_numerized'] = df_hdb.apply(flat_type_to_num, axis=1)


In [ ]:
df_hdb[['flat_type','flat_type_numerized']]

In [ ]:
df_hdb['flat_type'].value_counts()

In [ ]:
df_hdb['flat_type_numerized'].value_counts()

🔷**TASK**

Merge both the **df_fac** together **df_hdb**, so everyone record in **df_hdb** will have the corresponding information about the facilities.

Store the merged dataframe as a new variable called **df**.

You may need more than one operation to achieve this.

In [ ]:
df_hdb[['block','street_name']]

In [ ]:
df_fac[['address']]

In [ ]:
def create_address(row):
  blk = row['block']
  street = row['street_name']
  address= blk + ', ' + street
  return address

df_hdb['address'] =  df_hdb.apply(create_address, axis=1)


In [ ]:
df_hdb[['block','street_name','address']]


In [ ]:
df = pd.merge(df_hdb, df_fac, how='left', on='address')
df

🔷**TASK**

Filter to records of 2015 and onwards. We will use filtered **df_hdb** for the remaining of this notebook.


In [ ]:
df = df[(df_hdb['Tranc_Year'] >= 2015)]
df

---

# Exploratory Data Analysis

## Correlation

In [ ]:
matrix_corr = df[['Tranc_Year', 'floor_area_sqft', 'flat_type_numerized', 'resale_price', 'price_per_sqft']].corr()
matrix_corr

In [ ]:
sns.heatmap(matrix_corr, annot=True, cmap='Oranges')

### Breakdown and Understand the Pairplot

In [ ]:
sns.pairplot(df[['Tranc_Year', 'floor_area_sqft', 'flat_type_numerized', 'resale_price', 'price_per_sqft']])

### Scatterplot for Visualizing Relationship between 2 Variables

**NOTE**
- the seaborn's scatterplot can be used to easily visualize 3 dimensions (2 numerical, 1 categorical)
- the parameters 'x' and 'y' for the 2 numerical
- the parameter 'hue' uses the color of the dots to represent the categorical

In [ ]:
sns.scatterplot(x='floor_area_sqft', y='price_per_sqft', hue='flat_type', data=df)

## Descriptive Statistics

🔷**TASK**

Calculate the average **resale_price** of **4 ROOM** HDB units

In [ ]:
# Average of Height
df_four_room = df[df['flat_type'] == '4 ROOM']
avg_four_room_resaleprice= df_four_room['resale_price'].mean()
avg_four_room_resaleprice

🔷**TASK**

Use histogram to display the distribution of **resale_price** of all the **4 ROOM** hdb units.

In [ ]:
sns.histplot(data=df_four_room,
             x='resale_price')
plt.show()

In [ ]:
plt.hist(df_four_room['resale_price'])
plt.show()

🔷**TASK**

Use Boxplot (in seaborn) to display the distribution of **resale_price** of the **4 ROOM** hdb unit, for each of the **Tranc_Year**.


In [ ]:
sns.boxplot( x=df_four_room['Tranc_Year'], y=df_four_room['resale_price']);
plt.show()

---

# Modeling with Linear Regression

![ml_workflow](https://i.imgur.com/xf43rGO.png)

## Model Selection

In [ ]:
# These are common modules to import regardless the model
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics

In [ ]:
features = ['Tranc_Year', 'floor_area_sqft', 'flat_type_numerized']
target = 'resale_price'

## Separating Train and Test Data sets

In [ ]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(df[features], df[target], test_size=0.2, random_state=255)

![ml_coresteps](https://i.imgur.com/zmxg7Un.png)

### Key Step 1: Instantiate the Model

In [ ]:
# Import the sepcifc model type
from sklearn import linear_model

m1 = linear_model.LinearRegression()

### Key Step 2: Train the Model

In [ ]:
m1.fit(x_train, y_train)

### Key Step 3: Predict based on new inputs

In [ ]:
predictions = m1.predict(x_test)

### Key Step 4 (New): Validate the the Predicted Values

In [ ]:
mse = metrics.mean_squared_error(predictions, y_test) # <- y_test is the actual values
rmse = np.sqrt(mse)

print("About 95% of these predictions are between -" + str(np.round(2 * rmse, 2)) + " and " + str(np.round(2 * rmse, 2))
      + "of actual resale values")

### Key 5 (New): Understand the Predictors

This allow you to find what which are the more important features that contribute to the target (i.e. resale_price).

In many cases, you care less about the predictions, but more about which are the factors are more important, so you can prioritize or explain the phenomenon.

In [ ]:
coeffecients = pd.DataFrame(m1.coef_, features)
coeffecients.columns = ['Coeffecient']
coeffecients

Interpreting the coefficients:

⚠ Fill in the values in placeholders <..> below

- Holding all other features fixed, a 1 unit increase in **Tranc_Year** is associated with an **increase of <..> resale_price**.
- Holding all other features fixed, a 1 unit increase in **floor_area_sqft** is associated with an **increase of <..> resale_price**.
- Holding all other features fixed, a 1 unit increase in **flat_type_numerized** is associated with an **increase of <..> resale_price**.


# More Hands-on Practices: Build More Models

## Model B

🔷**TASK**

Train and Test a new Linear Regression model with more features.

In [ ]:
# replace all the missing values in the df dataframe with 0
df = df.fillna(0)

In [ ]:

# These are the features from previous model
features = ['Tranc_Year', 'floor_area_sqft', 'flat_type_numerized']
target = 'resale_price'

In [ ]:
# Train a new model with these additional features
features_additional = [
 'Mall_Nearest_Distance',
 'Mall_Within_1km',
 'Hawker_Nearest_Distance',
 'Hawker_Within_1km',
]

In [ ]:
features.extend(features_additional)
target = 'resale_price'

In [ ]:
# Make sure you use random_state=255 so we can have the same result
# Use 80% of the data for training and 20% for testing
x_train, x_test, y_train, y_test = model_selection.train_test_split(df[features], df[target], test_size=0.2, random_state=255)

In [ ]:
# Instantiate a Linear Regression Model and assign to the "m1" variable (no hyperparameter needed)
m2 = linear_model.LinearRegression()

# Train the model
m2.fit(x_train, y_train)

# Use the trained model to generate predictions based on the test data
predictions = m2.predict(x_test)

# Validate the model
mse = metrics.mean_squared_error(predictions, y_test)
rmse = np.sqrt(mse)

print("About 95% of these predictions are between -" + str(np.round(2 * rmse, 2)) + " and " + str(np.round(2 * rmse, 2))
      + " cm of actual values")

In [ ]:
# Understand the Predictors
coeffecients = pd.DataFrame(m2.coef_, features)
coeffecients.columns = ['Coeffecient']
coeffecients

In [ ]:
print('Holding all other features fixed, a 1 unit increase in Tranc_Year is associated with an increase of '  + str(coeffecients.loc['Tranc_Year']['Coeffecient']) + ' resale_price.')
print('Holding all other features fixed, a 1 unit increase in floor_area_sqft is associated with an increase of '  + str(coeffecients.loc['floor_area_sqft']['Coeffecient']) + ' resale_price.')
print('Holding all other features fixed, a 1 unit increase in flat_type_numerized is associated with an increase of '  + str(coeffecients.loc['flat_type_numerized']['Coeffecient']) + ' resale_price.')
print('Holding all other features fixed, a 1 unit increase in Mall_Nearest_Distance is associated with an increase of '  + str(coeffecients.loc['Mall_Nearest_Distance']['Coeffecient']) + ' resale_price.')
print('Holding all other features fixed, a 1 unit increase in Mall_Within_1km is associated with an increase of '  + str(coeffecients.loc['Mall_Within_1km']['Coeffecient']) + ' resale_price.')
print('Holding all other features fixed, a 1 unit increase in Hawker_Nearest_Distance is associated with an increase of '  + str(coeffecients.loc['Hawker_Nearest_Distance']['Coeffecient']) + ' resale_price.')
print('Holding all other features fixed, a 1 unit increase in Hawker_Within_1km is associated with an increase of '  + str(coeffecients.loc['Hawker_Within_1km']['Coeffecient']) + ' resale_price.')

---

## Model C

🔷**TASK**

Train and Test a new Linear Regression model with the same features, with the following requirements:
- Use all records before 2021 to predict those records of 2021
- In other words, you will be training your model with records from 2015 to 2020, to predict records from 2021.

Pay attention to the comments started **#** with for further requirements

---

In [ ]:
df_test = df[(df_hdb['Tranc_Year'] == 2021)] #Records from 2021
df_train = df[(df_hdb['Tranc_Year'] < 2021)]# Records from 2015 to 2020

In [ ]:
df_test.head()

In [ ]:
df_train.tail()

In [ ]:
df_test = df[df['Tranc_Year'] == 2021]
df_train = df[df['Tranc_Year'] < 2021]

In [ ]:
# Do not change this cell.
# Must execute this cell.
df_train = df_train.sample(len(df_train), random_state=255) # Shuffle the order of records in df_train

---

🔷**TASK**

Split the df_train and df_test into the required variables **x_train, y_train, x_test, y_test** to build the model

In [ ]:
x_train = df_train[features]
y_train = df_train[target]
x_test = df_test[features]
y_test = df_test[target]

---

In [ ]:
# Instantiate a Linear Regression Model and assign to the "m1" variable (no hyperparameter needed)
m1 = linear_model.LinearRegression()

# Train the model
m1.fit(x_train,y_train)

# Use the trained model to generate predictions based on the test data
predictions = m1.predict(x_test)

# Validate the model
mse = metrics.mean_squared_error(predictions, y_test)
rmse = np.sqrt(mse)

print("About 95% of these predictions are between -" + str(np.round(2 * rmse, 2)) + " and " + str(np.round(2 * rmse, 2))
      + " cm of actual values")

In [ ]:
# Understand the Predictors
coeffecients = pd.DataFrame(m1.coef_, features)
coeffecients.columns = ['Coeffecient']
coeffecients

---